Dieses Notebook basiert auf Material von: https://zilliz.com/learn/vector-index

In [1]:
!uv pip install scipy

Using Python 3.12.8 environment at: .py312
Audited 1 package in 3ms


# Import / Config

In [2]:
import numpy as np
from scipy.cluster.vq import kmeans2

np.set_printoptions(precision=3, threshold=20, suppress=True)

# Data Generation / Query Vector

In [3]:
np.random.seed(4711)
dataset = np.random.normal(size=(1000, 128))
dataset

array([[-0.253, -0.383, -0.406, ...,  0.18 , -0.448,  0.107],
       [-1.252,  0.165,  0.664, ..., -1.347,  0.531, -1.985],
       [ 1.798,  0.604, -0.341, ...,  1.271,  0.929,  2.086],
       ...,
       [-0.897,  0.119, -0.685, ..., -0.169,  0.857, -0.133],
       [ 1.79 , -0.518, -1.168, ..., -0.037,  1.248, -1.068],
       [-0.248, -0.075,  0.221, ..., -0.792,  1.625, -0.607]],
      shape=(1000, 128))

In [4]:
query = np.random.normal(size=(128,))
query

array([-1.27 , -0.859,  0.116, ...,  0.145, -1.174, -0.453], shape=(128,))

# Linear Search

In [5]:
# Funktionsweise von argmin im Vergleich zu min
# argmin ist der index des minimalen Wertes
l1, l2, l3 = [10, 20, 30], [40, 20, 30], [40, 50, 30]
print(f"{np.min(l1)} | {np.argmin(l1)}")
print(f"{np.min(l2)} | {np.argmin(l2)}")
print(f"{np.min(l3)} | {np.argmin(l3)}")

10 | 0
20 | 1
30 | 2


In [6]:
# np.linalg.norm berechnet die Länge des Vektors
# in diesem Falle die Länges des Differenzvektors
# Broadcating
dists = np.linalg.norm(dataset - query, axis=1)
print(dists.shape)
print(dists)

(1000,)
[17.374 15.793 17.305 ... 17.906 16.369 16.277]


In [7]:
nearest1 = np.argmin(dists)
# Index des nächstgelegenen Vektors
print(nearest1)
# Der Vektor selbst
print(dataset[nearest1])

211
[ 0.642 -1.374  0.165 ...  1.923 -1.046 -0.186]


# IVF Index - Inverted File Index

In [8]:
num_part = 16  # Anzah IVF Partitionen
(centroids, assignments) = kmeans2(dataset, num_part, iter=16, seed=102)
print(centroids.shape)
print(centroids)
print(assignments.shape)
print(assignments)

(16, 128)
[[ 0.406 -0.094 -0.433 ...  0.065  0.632 -0.446]
 [-0.466 -0.748 -0.098 ...  0.599 -0.347 -0.656]
 [ 0.188  0.082 -0.093 ...  0.157  0.176 -0.294]
 ...
 [-0.006  0.142  0.002 ... -0.34   0.042  0.747]
 [ 0.06   0.072  0.252 ...  0.21   0.068  0.203]
 [-0.188  0.186  0.183 ... -0.188  0.172 -0.143]]
(1000,)
[13  2 12 ... 15  2 10]


In [9]:
# Lege einen leeren Index an, d.h. num_part leere Partitionen
index = [[] for _ in range(num_part)]
print(len(index))
print(index)

# Ordne jeder Partition die Assigments zu
for n, k in enumerate(assignments):
    index[k].append(n)

# Zeige beispielhaft Partition 1 und 9
# Achtung: die Partitionen enhalten die Indizes der Vektoren
# nicht die Vektoren selbst
print(index[1])
print(index[9])

16
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[81, 276, 311, 470, 623, 653, 741, 839, 970]
[10, 21, 230, 349, 518, 545, 654, 842, 857, 916]


In [10]:
# Berechne den zur Query nächstgelegenen Zentroiden
c = np.argmin(np.linalg.norm(centroids - query, axis=1))

# dataset[index[c]] liefert einen 'Slice' aller Vektoren
# nämlich genau die, die in der Partition des Zentroiden liegen
# Die Berechnung der Abstände erfolgt nur im Slice, d.h. bzgl. eine Teilmenge der Vektoren
nearest2 = np.argmin(np.linalg.norm(dataset[index[c]] - query, axis=1))  
# Gib den zur Query nächstgelegenen Vektor aus,
# der sich aus der Nutzung des Indexes ergibt
print(dataset[index[c]][nearest2])
# Gibt echten nächstgelegenen Vektor aus
print(dataset[nearest1])
# Man sieht unteschiedliche Werte, d.h. mit dem Index wird nur eine Näherungslösung ermittelt
# Die Berechnung per Index ist aber erheblich schneller
# Erhöhe dataset von 1000 auf eine Millionen Vektoren

[ 0.708 -1.25  -0.341 ... -0.874 -2.145  1.198]
[ 0.642 -1.374  0.165 ...  1.923 -1.046 -0.186]


# Skalare Quantitisierung

In [11]:
# Berechne die minimalen Werte in jeder Vektordimensionen
mins = np.min(dataset, axis=0)
print(mins.shape)
print(mins)

# Berechne die maximalen Werte in jeder Vektordimensionen
maxs = np.max(dataset, axis=0)
print(maxs.shape)
print(maxs)

(128,)
[-3.016 -2.68  -3.892 ... -2.98  -3.526 -3.197]
(128,)
[3.414 4.067 3.279 ... 3.095 3.074 3.258]


In [12]:
# Berechnung Bin-Größe bei 255 Bins
bin_sizes = (maxs - mins) / 255
print(bin_sizes.shape)
print(bin_sizes)

(128,)
[0.025 0.026 0.028 ... 0.024 0.026 0.025]


In [13]:
# Verteilung der Werte in die Bins
# Die Vektoren haben jetzt uint8-Werte statt float64
# d.h. 8-fache Reduzierung der Größe
print(dataset.dtype)
dataset_q = np.uint8((dataset - mins) / bin_sizes)
print(dataset_q.dtype)
print(dataset_q.shape)
print(dataset_q)

float64
uint8
(1000, 128)
[[109  86 123 ... 132 118 130]
 [ 69 107 162 ...  68 156  47]
 [190 124 126 ... 178 172 208]
 ...
 [ 84 105 114 ... 118 169 121]
 [190  81  96 ... 123 184  84]
 [109  98 146 ...  91 199 102]]


In [14]:
# Test, dass die quantitisierten Werte tasächlich zwischen 0 und 255 liegen
# Der Wert 254 ergibt sich durch Ungenauigkeiten bei den floats
mins_q = np.min(dataset_q, axis=0)
print(mins_q.shape)
print(mins_q)
maxes_q = np.max(dataset_q, axis=0)
print(maxes_q.shape)
print(maxes_q)

(128,)
[0 0 0 ... 0 0 0]
(128,)
[255 254 255 ... 255 255 255]


In [15]:
# Quantitisierung des Query-Vektors
query_q = np.uint8((query - mins) / bin_sizes)
print(query_q.shape)
print(query_q)

(128,)
[ 69  68 142 ... 131  90 108]


In [16]:
# Berechnung des nächstgelegenen Vektors auf quantitisierter Ebene
dists2 = np.linalg.norm(dataset_q - query_q, axis=1)
nearest3 = np.argmin(dists2)
# Index des nächstgelegenen Vektors
print(nearest3)
# Der quantitisierte Vektor
print(dataset_q[nearest3])
# Der ursprüngliche Vektor
print(dataset[nearest3])
print(dataset[nearest1])
# Auch hier wieder nur Näherungslösung

56
[ 77  51 171 ... 171 180 191]
[-1.064 -1.306  0.933 ...  1.101  1.146  1.648]
[ 0.642 -1.374  0.165 ...  1.923 -1.046 -0.186]


# Produktquantitisierung

In [17]:
# Aufteilung in 16 Subvektoren
M = 16
# Jeder Subvektor wird in 256 Regionen geclustert
K = 128

In [18]:
# Anzahl Dimensionen pro Subvektor
sublen = dataset.shape[1] // M
print(sublen)

8


In [19]:
# Slice mit allen Zeilen und den ersten 8 Dimensionen
subspace0 = dataset[:,0:sublen]
print(subspace0.shape)
print(subspace0)

(1000, 8)
[[-0.253 -0.383 -0.406 ... -0.582  0.081 -1.409]
 [-1.252  0.165  0.664 ... -0.758 -1.243 -0.524]
 [ 1.798  0.604 -0.341 ...  0.43   0.938 -0.348]
 ...
 [-0.897  0.119 -0.685 ...  0.888  0.605 -0.038]
 [ 1.79  -0.518 -1.168 ...  1.287  0.895 -1.35 ]
 [-0.248 -0.075  0.221 ... -0.422 -1.678  0.521]]


In [20]:
# Slice mit allen Zeilen und den folgenden 8 Dimensionen
subspace1 = dataset[:,sublen:sublen*2]
print(subspace1.shape)
print(subspace1)

(1000, 8)
[[-3.339 -0.328 -0.64  ... -0.888 -2.056  0.795]
 [-0.201  0.092 -0.565 ... -0.421  0.499 -0.543]
 [ 0.214 -0.469  0.983 ... -0.292  0.166  0.142]
 ...
 [ 0.148 -1.431  2.002 ...  0.613 -0.47  -0.135]
 [ 0.84   0.906 -0.301 ... -1.594  0.109 -1.642]
 [-1.383  1.309  0.357 ...  0.176  1.215  1.314]]


In [21]:
# Clustering von subspace0
(centroids0, assignments0) = kmeans2(subspace0, 32, iter=16, seed=102)
print(centroids0.shape)
print(centroids0)
print(assignments0.shape)
print(assignments0)

(32, 8)
[[ 1.023 -0.186  0.745 ...  0.34  -0.756 -0.103]
 [-0.052 -1.218  0.307 ... -0.234 -0.597  1.24 ]
 [-0.332  0.362  0.24  ... -0.251 -0.774 -0.968]
 ...
 [ 0.19  -0.171  1.243 ... -0.449  0.055  0.574]
 [ 1.611  0.526 -0.02  ...  0.9    1.042 -0.481]
 [ 0.332  1.425 -0.058 ... -1.087 -0.043  1.067]]
(1000,)
[14  2 16 ... 14 30 21]
